# Lecture 11
## CNN Class Layers Ensemble
<hr/>

### make Class 
```python
class Model:
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        self._build_net()
        
    def _build_net(self):
        with tf.variable_scope(self.name):
            self.X = tf.placeholder(tf.float32, [None, 784])
            X_img = tf.reshape(self.X, [-1, 28, 28, 1])
            self.Y = tf.placeholder(tf.float32, [None, 10])
            
            W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
            
            ...
            
            
    def predict(self, x_test, keep_prop = 1.0):
        return self.sess.run(self.logits, feed_dict = {self.X: x_test, self.keep_prob: keep_prob})
    
    def get_accuracy(self, x_test, y_test, keep_prop = 1.0):
        return self.sess.run(self.accuracy, feed_dict = {self.X: x_test, self.Y: y_test, self.keep_prob: keep_prob})
    
    def train(self, x_data, y_data, keep_prop = 1.0):
        return self.sess.run([self.cost, self.optimizer], feed_dict = {self.X: x_test, self.keep_prob: keep_prob})
    
  
sess = tf.Session()
m1 = Model(sess, 'm1')

batch_size = 100
training_epochs = 10

sess.run(tf.global_variables_initializer())
print('started!')

for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        c, _ = m1.train(batch_xs, batch_ys)
        avg_cost += c / total_batch    
    
```

### tf.layers
```python
conv1 = tf.layers.conv2d(inputs = X_img, filters = 32, kernel_size = [3,3], padding = 'SAME', activation = tf.nn.relu)
pool1 = tf.layers.max_pooling2d(inputs = conv1, pool_size = [2,2], padding = 'SAME', strides= 2)
dropout1 = tf.layers.dropout(inputs = pool1, rate = 0.7, training = self.training)
```

## Ensemble
<hr/>

```python
models = []
num_models = 7
for m in range(num_models):
    models.append(Model(sess, 'Model' + str(m)))
    
sess.run(tf.global_variables_initializer())
print('started!')

for epoch in range(training_epochs):
    avg_cost_list = np.zeros(len(models))
    total_batch = int(mnist.train.num_examples / batch_size)
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        
        for m_idx, m in enumerate(models):
            c, _ = m.train(batch_xs, batch_ys)
            avg_cost_list[m_idx] += c /total_batch
            
    print('Epoch:','%04d'%(epoch + 1),'cost=',avg_cost_list)
    
print('Finished!')
```

```python
test_size = len(mnist.test.labels)
predictions = np.zeros(test_size * 10).reshape(test_size, 10)

for m_idx, m in enumerate(models):
    print(m_idx, 'Accuracy', m.get_accuracy(mnist.test.images, mnist.test.labels))
    p = m.predict(mnist.test.images)
    predictions += p
    
ensemble_correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.arg_max(mnist.test.labels, 1))
ensemble_accuracy = tf.reduce_mean(tf.cast(ensemble_correct_prediction, tf.float32))
print('Ensenble accuracy:', sess.run(ensemble_accuracy))
```
